In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_selector, ColumnTransformer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import os
os.chdir("D:/Training/Academy/ML(Python)/Cases/Glass_Identification")

In [ ]:
glass = pd.read_csv("Glass.csv")
X, y = glass.drop('Type', axis=1), glass['Type']
le = LabelEncoder()
y = le.fit_transform(y)
lr = LogisticRegression()
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=25)
results = cross_val_score(lr, X, y, scoring='accuracy', cv=kfold)

In [16]:
np.mean( results )

0.6125138427464009

In [ ]:
results = cross_val_score(lr, X, y, scoring='f1_macro', cv=kfold)

In [18]:
np.mean( results )

0.434438043245936

In [19]:
solvers = ['lbfgs','newton-cg','newton-cholesky','sag','saga']
Cs = np.linspace(0.001, 15, 20)
penalties = ['l2', None]
scores = []
for s in tqdm( solvers ):
    for p in penalties:
        for c in Cs:
            lr = LogisticRegression(penalty=p ,solver=s, C=c)
            results = cross_val_score(lr, X, y, scoring='f1_macro', cv=kfold)
            scores.append([s, p,c, np.mean(results)])
df_scores = pd.DataFrame(scores, columns=['solver','penalty','C','score'])

  0%|          | 0/5 [00:00<?, ?it/s]c:\Users\hp\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\hp\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/pr

In [20]:
df_scores.sort_values('score', ascending=False)

,solver,penalty,C,score
100,newton-cholesky,None,0.001000,0.595134
110,newton-cholesky,None,7.895211,0.595134
101,newton-cholesky,None,0.790421,0.595134
103,newton-cholesky,None,2.369263,0.595134
104,newton-cholesky,None,3.158684,0.595134
...,...,...,...,...
160,saga,l2,0.001000,0.087347
120,sag,l2,0.001000,0.087347
80,newton-cholesky,l2,0.001000,0.087347
40,newton-cg,l2,0.001000,0.087347


In [11]:
from sklearn.tree import DecisionTreeClassifier

In [21]:
depths = [None, 3, 4,5,6,7]
min_samples = [2, 10, 0.025, 0.01 ,0.05, 0.1]
min_leaf = [1, 10, 0.025, 0.01 ,0.05, 0.1]
scores = []
for d in tqdm( depths ):
    for s in min_samples:
        for f in min_leaf:
            dtc = DecisionTreeClassifier(random_state=25, 
                    max_depth=d, min_samples_split=s, min_samples_leaf=f)
            results = cross_val_score(dtc, X, y, scoring='f1_macro', cv=kfold)
            scores.append([d,s,f, np.mean(results)])
df_scores = pd.DataFrame( scores, columns=['depth','min-samples','min-leaves','score'] )
df_scores.sort_values('score', ascending=False)

100%|██████████| 6/6 [00:05<00:00,  1.04it/s]


,depth,min-samples,min-leaves,score
144,6.0,2.000,1.00,0.635469
162,6.0,0.010,1.00,0.635469
166,6.0,0.010,0.05,0.614503
178,6.0,0.100,0.05,0.614503
184,7.0,2.000,0.05,0.614503
...,...,...,...,...
63,3.0,0.050,0.01,0.448707
57,3.0,0.010,0.01,0.448707
39,3.0,2.000,0.01,0.448707
45,3.0,10.000,0.01,0.448707
